# Train a Deep Q Network with TF-Agents

adapted from : https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial

# Install dependancies

In [ ]:
!pip install git+https://github.com/glmcdona/LuxPythonEnvGym.git

In [ ]:
!apt-get install --yes libpython3.7

In [ ]:
!pip install kaggle-environments -U
!pip install tf-agents 
!pip install dm-reverb[tensorflow]

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import necessary modules

In [ ]:

from luxai2021.game.constants import LuxMatchConfigs_Default
from luxai2021.game.game import Game
from luxai2021.game.unit import Unit
from luxai2021.game.resource import Resource
from luxai2021.game.constants import Constants
import numpy as np
import tensorflow as tf
from luxai2021.game.constants import LuxMatchConfigs_Default
from luxai2021.game.game import Game
from luxai2021.game.unit import Unit
from luxai2021.game.city import City, CityTile
from luxai2021.game.actions import MoveAction, SpawnCityAction, TransferAction, PillageAction, Action, SpawnWorkerAction
from luxai2021.game.constants import Constants
from luxai2021.game.resource import Resource
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common
from tf_agents.typing import types
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.trajectories import TimeStep, StepType
from tf_agents.agents.ppo import ppo_agent
from tf_agents.networks.actor_distribution_rnn_network import ActorDistributionRnnNetwork
from tf_agents.networks.value_rnn_network import ValueRnnNetwork
import tf_agents.trajectories as ts
from typing import Any, Callable, Optional, Sequence, Text, Union, List
import numpy as np
import tensorflow as tf
from tf_agents.environments import tf_py_environment

from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common

import tensorflow as tf
from tf_agents.environments import tf_py_environment
from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common
#from pyluxenvironmentworkerscalar import PyLuxEnvironmentWorkerScalar

#from pyluxenvironmentworkerscalar import PyLuxEnvironmentWorkerScalar
#from observation_builder import build_observation, build_reward, get_unit, get_unit_adjacent
import reverb
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.replay_buffers import reverb_replay_buffer

from tf_agents.drivers import py_driver
from tf_agents.policies import py_tf_eager_policy

import matplotlib
import matplotlib.pyplot as plt

import os
from tf_agents.policies import policy_saver
import random
import time

In [ ]:
import reverb
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.replay_buffers import reverb_replay_buffer

# Utility functions

In [ ]:

def my_get_unit_adjacent(game : Game, team : int, unit_id, direction : Constants.DIRECTIONS) -> Unit:
    unit = my_get_unit(game, unit_id)
    if unit is None:
        print(42)
    new_pos = unit.pos.translate(direction, 1)
    new_cell = game.map.get_cell_by_pos(new_pos)
    all_units = list(game.state["teamStates"][team]["units"].values())
    unit_adjacent = [unit for unit in all_units if unit.pos.x == new_pos.x and unit.pos.y == new_pos.y]
    if len(unit_adjacent) == 0:
        return None
    elif len(unit_adjacent) == 1:
        return unit_adjacent[0]
    else:
        raise ValueError("my_get_unit_adjacent")
    return None


def my_get_unit(game : Game, unit_id : str) -> Unit:
    if unit_id in game.state['teamStates'][0]['units'].keys():
        return game.state['teamStates'][0]['units'][unit_id]
    elif unit_id in game.state['teamStates'][1]['units'].keys():
        return game.state['teamStates'][1]['units'][unit_id]
    else:
        return None

def my_get_units(game : Game, team : int) -> List[Unit]:
    return list(game.state['teamStates'][team]['units'].values())

def my_get_citytiles(game, team)  -> List[CityTile]  :
    city_tiles = []
    for city in game.cities.values():
        if city.team == team:
            for citycell in city.city_cells:
                city_tiles.append(citycell.city_tile)
    return city_tiles


# Reward function

In [ ]:

def build_reward(game : Game, unit_id : str, team : int) -> float:
    #global nbcall
    #nbcall += 1
    unit = my_get_unit(game, unit_id)
    #if not unit is None:
    #    print("unit pos ({}, {})".format(unit.pos.x, unit.pos.y))
    #if unit is None:
    #    print("unit {} None".format(unit_id))
    #else:
    #    print("unit {} ({}, {})".format(unit_id, unit.pos.x, unit.pos.y))
            
    ctt = my_get_citytiles(game, team)
    reward_ctt= 100 * (0 if ctt is None else len(ctt))
    reward_ctt = 0
    reward_cfv = 0.9 * (unit.get_cargo_fuel_value() if unit is not None else 0.0)
    #if (reward_cfv > 0):
    #    print("got cargo {}".format(unit.get_cargo_fuel_value()))
    reward_fg = game.stats['teamStats'][team]['fuelGenerated'] #game.state['turn']
    reward = -100.0 + reward_cfv + reward_fg + reward_ctt 
    #print("{} : {} ".format(nbcall, reward))
    #if nbcall == 198:
    #    print(42)
    return np.float32(reward)


# Create observation

In [ ]:

def build_observation(game : Game, unit_id : str):
    #unit = game.get_unit(team_id, unit_id)
    unit = my_get_unit(game, unit_id)
    
    if unit is None:
        return np.array([-1] * 259)
    team = unit.team
    
    other_team = 0 if team == 1 else 1
    
    game_state = [ game.state['turn'], 
                    1 if game.is_night() else 0,
                    1, # worker
                    0, # cart
                    0, # citytile,
                    unit.get_cargo_space_left(),
                    unit.cargo['wood'],
                    unit.cargo['coal'],
                    unit.cargo['uranium'],
                    unit.get_cargo_fuel_value(),                    
                    game.state['teamStates'][team]['researchPoints'],
                    1 if game.state['teamStates'][team]['researched']['wood'] else 0,
                    1 if  game.state['teamStates'][team]['researched']['coal'] else 0,
                    1 if game.state['teamStates'][team]['researched']['uranium'] else 0,
                    game.stats['teamStats'][team]['resourcesCollected']['wood'],
                    game.stats['teamStats'][team]['resourcesCollected']['coal'],
                    game.stats['teamStats'][team]['resourcesCollected']['uranium'],
                    game.stats['teamStats'][team]['cityTilesBuilt'],
                    game.stats['teamStats'][team]['workersBuilt'],
                    game.stats['teamStats'][team]['cartsBuilt'],
                    game.stats['teamStats'][team]['roadsBuilt'],
                    game.stats['teamStats'][team]['roadsPillaged'],
                    game.state['teamStates'][other_team]['researchPoints'],
                    1 if game.state['teamStates'][other_team]['researched']['wood'] else 0,
                    1 if game.state['teamStates'][other_team]['researched']['coal'] else 0,
                    1 if game.state['teamStates'][other_team]['researched']['uranium'] else 0,
                    game.stats['teamStats'][other_team]['resourcesCollected']['wood'],
                    game.stats['teamStats'][other_team]['resourcesCollected']['coal'],
                    game.stats['teamStats'][other_team]['resourcesCollected']['uranium'],
                    game.stats['teamStats'][other_team]['cityTilesBuilt'],
                    game.stats['teamStats'][other_team]['workersBuilt'],
                    game.stats['teamStats'][other_team]['cartsBuilt'],
                    game.stats['teamStats'][other_team]['roadsBuilt'],
                    game.stats['teamStats'][other_team]['roadsPillaged'] ]
    
    
    xs = np.array([2, 1, 0, -1, -2]) + unit.pos.x
    ys = np.array([2, 1, 0, -1, -2]) + unit.pos.y
    
    positions = []
    for y in ys:    
        for x in xs:
            if x < 0 or x >= game.map.width:
                positions.append(None)
            elif y < 0 or y >= game.map.height:
                positions.append(None)
            else:
                positions.append(game.map.get_cell(x, y))
    
    resources_wood = []
    resources_coal = []
    resources_uranium = []
    #resources_amounts = []
    
    for i in range(len(positions)):
        cell = positions[i]
        if cell is None or not cell.has_resource():
            #resources_types.append(0)
            resources_wood.append(0)
            resources_coal.append(0)
            resources_uranium.append(0)
        else:
            if cell.resource.type == Resource.Types.WOOD:
                resources_wood.append(cell.resource.amount)
                resources_coal.append(0)
                resources_uranium.append(0)
            elif cell.resource.type == Resource.Types.COAL:
                resources_wood.append(0)
                resources_coal.append(cell.resource.amount)
                resources_uranium.append(0)
            elif cell.resource.type == Resource.Types.URANIUM :
                resources_wood.append(0)
                resources_coal.append(0)
                resources_uranium.append(cell.resource.amount)
            else:
                raise ValueError("build_observation")
            
    
    city_tiles_team_mine = []
    city_tiles_team_other = []
    
    for i in range(len(positions)):
        cell = positions[i]
        if cell is None or not cell.is_city_tile():
            city_tiles_team_mine.append(0)
            city_tiles_team_other.append(0)
        else:
            ct = cell.city_tile
            if ct.team == team:
                city_tiles_team_mine.append(1)
                city_tiles_team_other.append(0)
            else:
                city_tiles_team_mine.append(0)
                city_tiles_team_other.append(1)

    worker_cargo_team_mine = [ -1 ] * len(positions)
    kart_cargo_team_mine = [ -1 ] * len(positions)
    worker_cargo_team_other = [ -1 ] * len(positions)
    kart_cargo_team_other = [ -1 ] * len(positions)
    
    for i in range(len(positions)):
        cell = positions[i]
        if cell is None or not cell.has_units():
            worker_cargo_team_mine[i] = -1
            kart_cargo_team_mine[i] = -1
            worker_cargo_team_other[i] = -1
            kart_cargo_team_other[i] = -1
        else:
            for unit in cell.units.values() :
                if unit.is_worker():
                    if unit.team == team:
                        worker_cargo_team_mine[i] = unit.cargo["wood"] + unit.cargo["coal"] + unit.cargo["uranium"]
                    else:
                        worker_cargo_team_other[i] = unit.cargo["wood"] + unit.cargo["coal"] + unit.cargo["uranium"]
                elif unit.is_cart():
                    if unit.team == team:
                        worker_cargo_team_mine[i] = unit.cargo["wood"] + unit.cargo["coal"] + unit.cargo["uranium"]
                    else:
                        worker_cargo_team_other[i] = unit.cargo["wood"] + unit.cargo["coal"] + unit.cargo["uranium"]
    
    observation = game_state + resources_wood + resources_coal + resources_uranium + city_tiles_team_mine + city_tiles_team_other + worker_cargo_team_mine + kart_cargo_team_mine + worker_cargo_team_other + kart_cargo_team_other
    return np.array(observation).astype(np.int32)



build the environment

# Define the RL environment

In [ ]:

class PyLuxEnvironmentWorkerScalar(py_environment.PyEnvironment):
    def __init__(self, worker_agent_class, kart_agent_class, city_agent_class, discount = np.float32(1.0)):
        self._configs = LuxMatchConfigs_Default
        self._game = Game(self._configs)
        self._episode_ended = False
        self._discount = discount
        self._current_time_step = None
        self._worker_agent_class = worker_agent_class
        self._kart_agent_class = kart_agent_class
        self._city_agent_class = city_agent_class
        self._worker_agent = None
        self._workers_to_manage = []
        self._carts_to_manage = []
        self._cities_to_manage = []
        self._action_spec = array_spec.BoundedArraySpec(shape=(),
                                                        dtype=np.int32,
                                                        minimum=0, 
                                                        maximum=10,
                                                        name='action')
        self._observation_spec = array_spec.ArraySpec(shape = (259,), dtype = np.int32)
        
    def _reset(self) -> TimeStep:
        if self._game is None or self._game.match_over():
            self._game = Game(self._configs)
            
         
        all_cities = list(self._game.cities.keys())
        all_unit_ids = list(self._game.state["teamStates"][0]["units"].keys()) + list(self._game.state["teamStates"][1]["units"].keys())
        all_workers = []
        all_carts = []
        
        for unit_id in all_unit_ids:
            unit = my_get_unit(self._game, unit_id)
            if unit.is_worker():
                all_workers.append(unit)
            elif unit.is_cart():
                all_carts.append(unit)
            else:
                raise ValueError("_reset1")
                
        if len(all_workers) > 0:
            self._carts_to_manage = all_carts
            self._worker_agent = np.random.choice(all_workers)
            self._worker_agent_team = self._worker_agent.team
            all_workers.remove(self._worker_agent)            
            self._workers_to_manage = all_workers # minus the one we just removed
            observation = build_observation(self._game, self._worker_agent.id)
            if not self._observation_spec.check_array(observation):
                #ipdb.set_trace()
                raise ValueError("PyLuxEnvironmentWorkerScalar._reset {}".format(type(observation)))
            step_type = StepType.FIRST
            reward = np.float32(-1.0)
            discount = self._discount
            first_ts = TimeStep(step_type, reward, discount, observation)
    
            self._current_time_step = first_ts
            self._episode_ended = False
            return self._current_time_step            
        else:
            #raise ValueError("_reset2")
            self._game = None
            return self._reset()
            
            
            

        
    def _step(self, action):
        if not self._action_spec.check_array(action):
            raise ValueError("PyLuxEnvironmentWorkerScalar._step {}".format(action))
        #print(action)
        turn = self._game.state['turn']
        #if turn == 198 or turn == 199:
        #    print("turn_step {}".format(turn))
        if self._episode_ended:
            #print("step reset")
            return self.reset()
        
        actions = [] 
        # Array of actions for both teams. Eg: MoveAction(team, unit_id, direction)        
        # game_over = game.run_turn_with_actions(actions)
        # observation, reward, self._episode_ended, self._info = self._game.run_turn_with_actions(actions)
        actions_spawn_workers = self.scalar_to_action_citytile(self._game, 42)
        unit = my_get_unit(self._game, self._worker_agent.id)
        if unit is not None:
            action_str = self.scalar_to_action_worker(action)
            if not action_str is None:
                actions.append(action_str)
        actions.extend(actions_spawn_workers)
        
        self._episode_ended = self._game.run_turn_with_actions(actions)
        if self._game.state['turn'] > 360:
            print("PyLuxEnvironmentWorkerScalar._step: turn > 360")
        team = self._worker_agent.team
        team_units = self._game.state['teamStates'][team]['units']
        if not self._worker_agent.id in team_units:
            #print("agent died")                        
            if len(team_units) == 0:
                team_ctt = my_get_citytiles(self._game, team)
                if len(team_ctt) == 0:
                    # end game as no city and no agent
                    self._episode_ended = True
                else:                    
                    bas = self.get_action_build_worker(self._game, team)
                    self._episode_ended = self._game.run_turn_with_actions(bas)
                    if not self._episode_ended:
                        team_workers = [ unit for unit in self._game.state['teamStates'][team]['units'].values() if unit.is_worker()]
                        if len(team_workers) == 0:
                            self._episode_ended = True
                        else:
                            worker = random.sample(team_workers, 1)[0]
                            self._worker_agent = worker
            else:
                team_workers = [ unit for unit in self._game.state['teamStates'][team]['units'].values() if unit.is_worker()]
                if len(team_workers) == 0:
                    self._episode_ended = True
                else:
                    worker = random.sample(team_workers, 1)[0]
                    self._worker_agent = worker
                
                    
            
        observation = build_observation(self._game, self._worker_agent.id)
        observation = np.array(observation , dtype=np.int32)
        reward = build_reward(self._game, self._worker_agent.id, self._worker_agent_team)
        if not self._observation_spec.check_array(observation):
            ipdb.set_trace()
            raise ValueError("PyLuxEnvironmentWorkerScalar._step")
        if self._episode_ended:# or self._worker_agent.id not in all_unit_ids:
            return ts.termination(observation, reward)
        else:
            #ipdb.set_trace()
            return ts.transition(observation, reward, self._discount)

    def scalar_to_action_citytile(self, game : Game, action : np.int32) -> List[Action] :
        # dummy to just generate more workers
        team0 = 0
        team1 = 1
        units0 = my_get_units(game, team0)
        units1 = my_get_units(game, team1)
        actions = []
        if len(units0) < 1:
            actionlist0 = self.get_action_build_worker(game, team0)
            actions.append(actionlist0)
        if len(units1) < 1:
            actionlist1 = self.get_action_build_worker(game, team0)
            actions.append(actionlist1)
        action_list = [item for sublist in actions for item in sublist]
        return action_list
            
    def get_action_build_worker(self, game : Game, team : int) -> List[Action] :
        ctt_list = [ctt for ctt in my_get_citytiles(game, team) if ctt.can_act() ]
        if len(ctt_list) == 0:
            return []
        ctt_list = random.sample(ctt_list, min(1, len(ctt_list)))
        actions = []
        for ctt in ctt_list:
            action = SpawnWorkerAction(team, None, ctt.pos.x, ctt.pos.y)
            actions.append(action)
        return actions
        
        
    def scalar_to_action_worker(self, action) -> Action :
        # add type checking
        unit_id = self._worker_agent.id
        team = self._worker_agent.team
        unit = my_get_unit(self._game, self._worker_agent.id)
        if unit is None:
            return None
        # dont move
        if action == 0:
            ma = MoveAction(team, unit_id, Constants.DIRECTIONS.CENTER)
            return ma
        # move north
        elif action == 1:
            ma = MoveAction(team, unit_id, Constants.DIRECTIONS.NORTH)
            return ma
        # move east
        elif action == 2:
            ma = MoveAction(team, unit_id, Constants.DIRECTIONS.EAST)
            return ma
        # move south
        elif action == 3:
            ma = MoveAction(team, unit_id, Constants.DIRECTIONS.SOUTH)
            return ma
        # move west
        elif action == 4:
            ma = MoveAction(team, unit_id, Constants.DIRECTIONS.WEST)
            return ma
        # build city
        elif action == 5:
            sca = SpawnCityAction(team, unit_id)
            return sca
        # pillage
        elif action == 6:
            pa = PillageAction(team, unit_id)
            return pa
        # transfer north
        elif action == 7:
            dest_unit = my_get_unit_adjacent(self._game, team, unit_id, Constants.DIRECTIONS.NORTH)
            if dest_unit is None:
                return None
            else:
                cargo_type = max(self._worker_agent.cargo, key=self._worker_agent.cargo.get)
                cargo_amount = self._worker_agent.cargo[cargo_type]
                ta = TransferAction(team, unit_id, dest_unit.id, cargo_type, cargo_amount)
                return ta
        # transfer east
        elif action == 8:
            dest_unit = my_get_unit_adjacent(self._game, team, unit_id, Constants.DIRECTIONS.EAST)
            if dest_unit is None:
                return None
            else:
                cargo_type = max(self._worker_agent.cargo, key=self._worker_agent.cargo.get)
                cargo_amount = self._worker_agent.cargo[cargo_type]
                ta = TransferAction(team, unit_id, dest_unit.id, cargo_type, cargo_amount)
                return ta
        # transfer south
        elif action == 9:
            dest_unit = my_get_unit_adjacent(self._game, team, unit_id, Constants.DIRECTIONS.SOUTH)
            if dest_unit is None:
                return None
            else:
                cargo_type = max(self._worker_agent.cargo, key=self._worker_agent.cargo.get)
                cargo_amount = self._worker_agent.cargo[cargo_type]
                ta = TransferAction(team, unit_id, dest_unit.id, cargo_type, cargo_amount)
                return ta
        # transfer west
        elif action == 10:
            dest_unit = my_get_unit_adjacent(self._game, team, unit_id, Constants.DIRECTIONS.WEST)
            if dest_unit is None:
                return None
            else:
                cargo_type = max(self._worker_agent.cargo, key=self._worker_agent.cargo.get)
                cargo_amount = self._worker_agent.cargo[cargo_type]
                ta = TransferAction(team, unit_id, dest_unit.id, cargo_type, cargo_amount)
                return ta
        else:
            raise ValueError("invalid action {}".format(action))
        
    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec


# Hyperparameters

In [ ]:
#DQNWorkerScalar
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 1  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 1000  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 200  # @param {type:"integer"}



In [ ]:
train_py_env = PyLuxEnvironmentWorkerScalar(str, str, str)
eval_py_env = PyLuxEnvironmentWorkerScalar(str, str, str)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)


# Define a Deep Q Network

In [ ]:


fc_layer_params = (259, 50)
action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
    return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])


optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

global_step = tf.compat.v1.train.get_or_create_global_step()
#train_checkpointer.initialize_or_restore()
#global_step = tf.compat.v1.train.get_global_step()
#saved_policy = tf.saved_model.load(policy_dir)

train_step_counter = tf.Variable(0)
epoch_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)


# Test the Random policy

In [ ]:

agent.initialize()

eval_policy = agent.policy
collect_policy = agent.collect_policy

from tf_agents.policies import random_tf_policy
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

    total_return = 0.0
    
    for _ in range(num_episodes):

        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]





In [ ]:
avg_random = compute_avg_return(eval_env, random_policy, num_eval_episodes)

In [ ]:
avg_random

# Replay Buffer

In order to keep track of the data collected from the environment, we will use [Reverb](https://deepmind.com/research/open-source/Reverb), an efficient, extensible, and easy-to-use replay system by Deepmind. It stores experience data when we collect trajectories and is consumed during training.

This replay buffer is constructed using specs describing the tensors that are to be stored, which can be obtained from the agent using agent.collect_data_spec.

In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

# Data Collection

Now execute the random policy in the environment for a few steps, recording the data in the replay buffer.

Here we are using 'PyDriver' to run the experience collecting loop. You can learn more about TF Agents driver in our [drivers tutorial](https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial).

In [ ]:
env = PyLuxEnvironmentWorkerScalar(str, str, str)
#@test {"skip": true}
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(train_py_env.reset())

The agent needs access to the replay buffer. This is provided by creating an iterable `tf.data.Dataset` pipeline which will feed data to the agent.

Each row of the replay buffer only stores a single observation step. But since the DQN Agent needs both the current and next observation to compute the loss, the dataset pipeline will sample two adjacent rows for each item in the batch (`num_steps=2`).

This dataset is also optimized by running parallel calls and prefetching data.

In [ ]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

In [ ]:
iterator = iter(dataset)
#print(iterator)

# Training the agent

Two things must happen during the training loop:

-   collect data from the environment
-   use that data to train the agent's neural network(s)

This example also periodicially evaluates the policy and prints the current score.

The following will take ~5 minutes to run.

In [ ]:
#@test {"skip": true}
try:
    %%time
except:
    pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = train_py_env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

start_time = time.time()

for _ in range(num_iterations):

    
    # Collect a few steps and save to the replay buffer.
    time_step, _ = collect_driver.run(time_step)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss

    step = agent.train_step_counter.numpy()

    if step % log_interval == 0:
        print('step = {0:,d}: loss = {1:,d}'.format(step, int(train_loss)))

    if step % eval_interval == 0:
        
        avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
        print('step = {0:,d}: Average Return = {1:,d}'.format(step, int(avg_return)))
        returns.append(avg_return)
        #print("--- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()
    
print("done")

# Plot the training improvements

In [ ]:
#@test {"skip": true}

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
#plt.ylim(top=250)
plt.show()